# Enhanced Barabási-Albert Network Generation

### Enhanced Preferential Attachment

Our implementation extends the classic BA model by introducing a power parameter to modify the preferential attachment mechanism:

$$P(k_i) \propto k_i^{\alpha}$$

Where:
- $P(k_i)$ is the probability of connecting to node $i$
- $k_i$ is the degree of node $i$
- $\alpha$ is the preferential attachment power

   - Create a fully connected initial subgraph with $m_0$ nodes
   - Establish a basic connectivity structure
   - For each new node $j$:
     a. Calculate powered degrees: $\text{degree}(i)^{\alpha}$
     b. Normalize probabilities
     c. Probabilistically select $m$ target nodes to connect


In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import time


def baGraph(m0, m, N, power=1):
    G = nx.complete_graph(m0)
    metrics = {
        'clustering': [nx.average_clustering(G)],
        'path_length': [nx.average_shortest_path_length(G)],
        'degree_distribution': [dict(G.degree())]
    }

    for i in range(m0, N):
        degrees = dict(G.degree())

        wtgDeg = {node: degree **power for node, degree in degrees.items()}
        totalWeight = sum(wtgDeg.values())
        probabilities = {node: weight/totalWeight for node,weight in wtgDeg.items()}

        targets = np.random.choice(list(degrees.keys()),size=m,replace=False,p=list(probabilities.values()))

        G.add_node(i)
        for target in targets:
            G.add_edge(i, target)
        try:
            metrics['clustering'].append(nx.average_clustering(G))
            metrics['path_length'].append(nx.average_shortest_path_length(G))
            metrics['degree_distribution'].append(dict(G.degree()))
        except nx.NetworkXError:
            metrics['clustering'].append(nx.average_clustering(G))
            metrics['path_length'].append(float('inf'))
            metrics['degree_distribution'].append(dict(G.degree()))

    return G, metrics


def instance(m0, m, N, powers=[1, 2, 3], num=100):
    summ = {}

    for a in m0:
        for b in m:
            for c in N:
                if b > a:
                    continue
                for p in powers:
                    key = f"m0={a}, m={b}, N={c}, power={p}"
                    print(f"Running {key}...")

                    results = {'clustering': np.zeros(c - a + 1),'path_length': np.zeros(c - a + 1),'final_degree_dist': []
                    }

                    for i in range(num):
                        if i % 10 == 0:
                            print(f"  Instance {i}/{num}")
                        G, metrics = baGraph(a, b, c, power=p)
                        results['clustering'] += np.array(metrics['clustering'])
                        results['path_length'] += np.array(metrics['path_length'])
                        finalDeg = metrics['degree_distribution'][-1]
                        results['final_degree_dist'].append(finalDeg)

                    results['clustering'] /=num
                    results['path_length']/= num

                    all_degrees = []
                    for dist in results['final_degree_dist']:
                        all_degrees.extend(list(dist.values()))

                    degcnt = Counter(all_degrees)
                    totalNodes = sum(degcnt.values())

                    results['degree_dist'] = {k: count/totalNodes for k, count in degcnt.items()}
                    summ[key] = results

    return summ


def plot(results):
    param_groups = {}
    for key in results.keys():
        base_key = key.rsplit(', power=', 1)[0]
        if base_key not in param_groups:
            param_groups[base_key] = []
        param_groups[base_key].append(key)

    for base_key, param_keys in param_groups.items():

        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        params_dict = dict(item.split('=') for item in base_key.split(', '))
        m0, m, N = int(params_dict['m0']), int(
            params_dict['m']), int(params_dict['N'])

        colors = ['b', 'r', 'g', 'purple', 'orange', 'brown']

        for i, key in enumerate(param_keys):
            power = float(key.split('power=')[1])
            data = results[key]
            axes[0].plot(range(m0, N+1), data['clustering'],
                         label=f'power={power}', color=colors[i % len(colors)])

        axes[0].set_title(f'Average Clustering Coefficient\n{base_key}')
        axes[0].set_xlabel('Network Size')
        axes[0].set_ylabel('Clustering Coefficient')
        axes[0].grid(True)
        axes[0].legend()

        for i, key in enumerate(param_keys):
            power = float(key.split('power=')[1])
            data = results[key]
            valid_path_lengths = data['path_length'][data['path_length'] < float(
                'inf')]
            if len(valid_path_lengths) > 0:
                axes[1].plot(range(m0, m0 + len(valid_path_lengths)), valid_path_lengths,
                             label=f'power={power}', color=colors[i % len(colors)])

        axes[1].set_title(f'Characteristic Path Length\n{base_key}')
        axes[1].set_xlabel('Network Size')
        axes[1].set_ylabel('Path Length')
        axes[1].grid(True)
        axes[1].legend()

        for i, key in enumerate(param_keys):
            power = float(key.split('power=')[1])
            data = results[key]
            degrees = sorted(data['degree_dist'].keys())
            probabilities = [data['degree_dist'][k] for k in degrees]

            axes[2].loglog(degrees, probabilities, 'o-',
                           label=f'power={power}', color=colors[i % len(colors)])

        axes[2].set_title(f'Degree Distribution (log-log)\n{base_key}')
        axes[2].set_xlabel('Degree (k)')
        axes[2].set_ylabel('P(k)')
        axes[2].grid(True, which='both', linestyle='--', alpha=0.7)
        axes[2].legend()

        plt.tight_layout()
        plt.savefig(
            f'baNetwork_variants_{base_key.replace(", ", "_").replace("=", "")}.png')
        plt.close(fig)


def main():
    m0 = [5, 10]
    m = [1, 2, 3]
    N = [100, 200]

    powers = [1, 2, 3]

    start = time.time()
    results = instance(m0=m0, m=m, N=N, powers=powers, num=100)
    plot(results)
    end = time.time()
    print(f"Total execution time: {end - start:.2f} seconds")


if __name__ == "__main__":
    main()

Running m0=5, m=1, N=100, power=1...
  Instance 0/100
  Instance 10/100
  Instance 20/100
  Instance 30/100
  Instance 40/100
  Instance 50/100
  Instance 60/100
  Instance 70/100
  Instance 80/100
  Instance 90/100
Running m0=5, m=1, N=100, power=2...
  Instance 0/100
  Instance 10/100
  Instance 20/100
  Instance 30/100
  Instance 40/100
  Instance 50/100
  Instance 60/100
  Instance 70/100
  Instance 80/100
  Instance 90/100
Running m0=5, m=1, N=100, power=3...
  Instance 0/100
  Instance 10/100
  Instance 20/100
  Instance 30/100
  Instance 40/100
  Instance 50/100
  Instance 60/100
  Instance 70/100
  Instance 80/100
  Instance 90/100
Running m0=5, m=1, N=200, power=1...
  Instance 0/100
  Instance 10/100
  Instance 20/100
  Instance 30/100
  Instance 40/100
  Instance 50/100
  Instance 60/100
  Instance 70/100
  Instance 80/100
  Instance 90/100
Running m0=5, m=1, N=200, power=2...
  Instance 0/100
  Instance 10/100
  Instance 20/100
  Instance 30/100
  Instance 40/100
  Instance

- Comparison with different $\alpha$ values are being plotted alongwith the classic BA model for comparison. The main difference is that while increasing the attachement power, the network's hub becomes more dense and the network becomes more centralized. In this sense, the network follows rich gets richer and poor gets poorer principle more strictly. 